## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-18-11-05-49 +0000,nyt,Talks on Russia-Ukraine War Enter 2nd Day in G...,https://www.nytimes.com/2026/02/18/world/europ...
1,2026-02-18-11-05-05 +0000,bbc,Social media suspended in Gabon for 'spreading...,https://www.bbc.com/news/articles/cx2rg2vndk1o...
2,2026-02-18-11-04-00 +0000,wsj,Auto & Transport Roundup: Market Talk,https://www.wsj.com/business/auto-transport-ro...
3,2026-02-18-11-00-00 +0000,wapo,How the Rev. Jesse Jackson taught me to keep h...,https://www.washingtonpost.com/business/2026/0...
4,2026-02-18-11-00-00 +0000,nypost,Saks and Neiman low on luxury goods as fashion...,https://nypost.com/2026/02/18/business/saks-an...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2553/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,28
22,jackson,27
21,jesse,23
110,new,18
0,talks,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
210,2026-02-17-21-36-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...,97
5,2026-02-18-11-00-00 +0000,nypost,Lara Trump reveals Trump’s unexpected reaction...,https://nypost.com/2026/02/18/us-news/lara-tru...,91
135,2026-02-18-00-48-22 +0000,bbc,"Under pressure from Trump, Venezuela's new pre...",https://www.bbc.com/news/articles/cn87rv0jdy1o...,89
268,2026-02-17-18-23-29 +0000,nyt,"Al Sharpton, Bernice King and Trump Among Trib...",https://www.nytimes.com/2026/02/17/us/politics...,86
367,2026-02-17-11-14-34 +0000,latimes,"Jesse Jackson, one of the nation's most powerf...",https://www.latimes.com/obituaries/story/2026-...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
210,97,2026-02-17-21-36-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...
367,78,2026-02-17-11-14-34 +0000,latimes,"Jesse Jackson, one of the nation's most powerf...",https://www.latimes.com/obituaries/story/2026-...
347,75,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...
62,51,2026-02-18-07-07-38 +0000,nypost,Mamdani’s ‘cuckoo’ property tax hike will land...,https://nypost.com/2026/02/18/us-news/mamdanis...
251,50,2026-02-17-19-34-19 +0000,latimes,Stephen Colbert calls out CBS for blocking int...,https://www.latimes.com/entertainment-arts/bus...
313,49,2026-02-17-16-02-41 +0000,nypost,"Iran, US agree on ‘guiding principles’ of nucl...",https://nypost.com/2026/02/17/world-news/iran-...
273,49,2026-02-17-18-14-06 +0000,nyt,Ukraine and Russia Hold New Round of Peace Tal...,https://www.nytimes.com/2026/02/17/world/europ...
82,47,2026-02-18-04-50-10 +0000,bbc,Suspect in Rhode Island ice hockey shooting ki...,https://www.bbc.com/news/articles/cy4wvk9vn40o...
69,44,2026-02-18-06-20-40 +0000,nypost,Skiers trapped by avalanche up Lake Tahoe moun...,https://nypost.com/2026/02/18/us-news/skiers-t...
354,39,2026-02-17-12-23-37 +0000,nypost,Top-performing fund warns software firms face ...,https://nypost.com/2026/02/17/business/top-per...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
